In [2]:
from enum import Enum
import random
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from sklearn import metrics
import numpy

In [2]:
class NodeType(Enum):
    USER = 0
    MOVIE = 1

node_map = []
graph = []
test_graph = []
test_size = 0.05

user_map = {}
movie_map = {}

last_movie_idx = 0
with open("combined_data_1.txt") as data_file:
    for line in tqdm(data_file):
        line = line.strip()
        if line.endswith(":"):
            movie_id = int(line[:-1])
                
            movie_map[movie_id] = len(node_map)
            last_movie_idx = len(node_map)
            
            for _ in range(5):
                node_map.append((NodeType.MOVIE, movie_id))
                graph.append([])
                test_graph.append([])
            
        else:
            node_info = line.split(",")
            user_id = int(node_info[0])
            rating = int(node_info[1]) - 1
            
            if user_id < 1200000:
                if user_id not in user_map.keys():
                    node_map.append((NodeType.USER, user_id))
                    graph.append([])
                    test_graph.append([])
                    
                    user_map[user_id] = len(node_map) - 1
                    
                user_node_idx = user_map[user_id]
                
                if random.uniform(0, 1) > test_size:
                    graph[last_movie_idx + rating].append(user_node_idx)
                    graph[user_node_idx].append(last_movie_idx + rating)
                else:
                    test_graph[last_movie_idx + rating].append(user_node_idx)
                    test_graph[user_node_idx].append(last_movie_idx + rating)

In [3]:
len(user_map)

213543

In [4]:
epochs = 10
t = 30
w = 8
latent_dim = 64

corpus = []

nodes = list(range(len(node_map)))
for _ in tqdm(range(epochs), total = epochs):
    random.shuffle(nodes)
    for node in nodes:
        walk = [node]
        for i in range(1, t):
            if len(graph[walk[-1]]) == 0:
                break
            walk.append(random.choice(graph[walk[-1]]))
        corpus.append([str(word) for word in walk])

In [5]:
model = Word2Vec(corpus, size = latent_dim, window = w, min_count = 0, sg = 1, hs = 1, workers = 4)

In [6]:
y_predicted = []
y_real = []
N = len(node_map)
print('ako ovo ispise znaci da je roundovao sve kako treba')
for user, node in tqdm(enumerate(node_map), total = N):
    
    if node[0] == NodeType.USER:
        
        similar_users = model.wv.most_similar(positive = [str(user)], topn = 50)
        
        for movie_rating_node in test_graph[user]:
            movie_node = movie_map[node_map[movie_rating_node][1]]
            
            rating_sum = 0
            rating_cnt = 0
            for similar_user in similar_users:
                similar_user_id = int(similar_user[0])
                for rating in range(5):
                    if (movie_node + rating) in graph[similar_user_id]:
                        rating_sum += rating + 1
                        rating_cnt += 1
                        
            if rating_cnt > 0:
                predicted_rating = round(rating_sum / rating_cnt)
            else:
                rating_sum = 0
                rating_cnt = 0
                for rating in range(5):
                    rating_sum += len(graph[movie_node + rating]) * (rating + 1)
                    rating_cnt += len(graph[movie_node + rating])
                if rating_cnt > 0:
                    predicted_rating = round(rating_sum / rating_cnt)
                else:
                    predicted_rating = 4
            real_rating = movie_rating_node - movie_node + 1
            
            y_predicted.append(predicted_rating)
            y_real.append(real_rating)

In [4]:
model = Word2Vec.load('deepwalk.model')

In [5]:
def max_k_off(y_real, y_predicted, k = 1):
    return len([i for i in range(len(y_real)) if abs(y_real[i] - y_predicted[i]) <= 1]) / len(y_real)

print('Model score:')
print(metrics.mean_absolute_error(y_real, y_predicted))
print(metrics.mean_squared_error(y_real, y_predicted))
print(metrics.r2_score(y_real, y_predicted))
print(max_k_off(y_real, y_predicted))
print('------------------')

y_constant4 = [4 for i in range(len(y_real))]

print('Constant 4 model score:')
print(metrics.mean_absolute_error(y_real, y_constant4))
print(metrics.mean_squared_error(y_real, y_constant4))
print(metrics.r2_score(y_real, y_constant4))
print(max_k_off(y_real, y_constant4))
print('------------------')

y_custom = [max(1, min(5, y + round(numpy.random.normal(0, 1.11)))) for y in y_real]
print('Normal distribution random model score:')
print(metrics.mean_absolute_error(y_real, y_custom))
print(metrics.mean_squared_error(y_real, y_custom))
print(metrics.r2_score(y_real, y_custom))
print(max_k_off(y_real, y_custom))
print('------------------')

y_allavg = []
for user, node in enumerate(node_map):
    if node[0] == NodeType.USER:
        for movie_rating_node in test_graph[user]:
            movie_node = movie_map[node_map[movie_rating_node][1]]
            
            rating_sum = 0
            rating_cnt = 0
            for rating in range(5):
                rating_sum += len(graph[movie_node + rating]) * (rating + 1)
                rating_cnt += len(graph[movie_node + rating])
            if rating_cnt > 0:
                predicted_rating = round(rating_sum / rating_cnt)
            else:
                predicted_rating = 4
            
            y_allavg.append(predicted_rating)

print('All average model score:')
print(metrics.mean_absolute_error(y_real, y_allavg))
print(metrics.mean_squared_error(y_real, y_allavg))
print(metrics.r2_score(y_real, y_allavg))
print(max_k_off(y_real, y_allavg))
print('------------------')

Model score:


NameError: name 'y_real' is not defined

In [15]:
model.save('deepwalk.model')